In [17]:
! pip install Flask


In [18]:
! pip install import_ipynb

In [19]:
! pip install pymysql

In [20]:
! pip install WTForms

In [21]:
! pip install mysqlclient

In [22]:
! pip install flask-mysqldb


In [23]:
! pip install flask-login

In [24]:
! pip freeze

asttokens==2.4.1
attrs==24.2.0
blinker==1.8.2
click==8.1.7
colorama==0.4.6
comm==0.2.2
debugpy==1.8.7
decorator==5.1.1
dnspython==2.7.0
email_validator==2.2.0
executing==2.1.0
fastjsonschema==2.21.0
Flask==3.0.3
Flask-Login==0.6.3
Flask-MySQLdb==2.0.0
idna==3.10
import-ipynb==0.2
ipykernel==6.29.5
ipython==8.29.0
itsdangerous==2.2.0
jedi==0.19.1
Jinja2==3.1.4
jsonschema==4.23.0
jsonschema-specifications==2024.10.1
jupyter_client==8.6.3
jupyter_core==5.7.2
MarkupSafe==3.0.2
matplotlib-inline==0.1.7
mysql-connector==2.2.9
mysql-connector-python==9.1.0
mysql-connector-python-rf==2.2.2
mysqlclient==2.2.6
nbformat==5.10.4
nest-asyncio==1.6.0
packaging==24.1
parso==0.8.4
platformdirs==4.3.6
prompt_toolkit==3.0.48
psutil==6.1.0
pure_eval==0.2.3
Pygments==2.18.0
PyMySQL==1.1.1
python-dateutil==2.9.0.post0
pywin32==308
pyzmq==26.2.0
referencing==0.35.1
rpds-py==0.21.0
six==1.16.0
stack-data==0.6.3
tornado==6.4.1
traitlets==5.14.3
wcwidth==0.2.13
Werkzeug==3.0.6
WTForms==3.2.1


In [25]:
from flask import Flask, request, render_template, redirect, url_for, jsonify, flash
import import_ipynb
import forms, hashlib 
from flask_mysqldb import MySQL

In [26]:
# intilize flask application
app = Flask(__name__)

In [27]:
app.config["SECRET_KEY"]="Hola como etas muy bein y tu?"
app.config['MYSQL_HOST']="localhost"
app.config["MYSQL_USER"]="root"
app.config["MYSQL_PASSWORD"]=""
app.config["MYSQL_DB"]="schoolapp"

In [28]:
mysql = MySQL(app)

In [29]:
@app.route("/")
def home():
    return redirect(url_for("login"))

In [30]:
@app.route("/login",methods=["GET","POST"])
def login():
    if request.method=="POST":
        login_form=forms.LoginForm(request.form)
        if login_form.validate():
            user_data=request.form
            email=user_data["email"]
            password=user_data["password"]
            hash=password+app.secret_key
            hash=hashlib.sha1(hash.encode())
            password=hash.digest()
            sql="select * from users where email=%s"
            cursor=mysql.connection.cursor()
            cursor.execute(sql,(email,))
            account = cursor.fetchone()
            if account:
                return "account exist"
            else :
                return "Invalid login credentials"
            cursor.close()
            return jsonify(login_form.data)
        else :
            return jsonify({"errors": login_form.errors}),400
    else :
        return render_template("auth/user.html")

In [31]:

@app.route("/register",methods=["GET","POST"])
def register():
    if request.method=="POST":
        print(request.form)
        register_form=forms.RegistrationForm(request.form)
        print(register_form.data)
        if register_form.validate():
            # return jsonify(register_form.data)
            user_data= request.form
            first_name=user_data["first_name"]
            last_name=user_data["last_name"]
            middle_name=user_data["middle_name"]
            email=user_data["email"]
            password=user_data["password"]
            phone_number=user_data["phone_number"]
            hash=password+app.secret_key
            hash=hashlib.sha1(hash.encode())
            password=hash.digest()
            cursor=mysql.connection.cursor()
            cursor.execute("select email from users where email=%s",(email,))

            account = cursor.fetchone()
            if account:
                return "account exist"
            else: 
                cursor.execute("insert into users ( `first_name`, `last_name`, `middle_name`, `email`, `password`, `gender`, `mobile_number`, `whatsapp_number`, `address`, `city`, `state`, `country`, `profile_picture`, `role`, `status`)values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)",(first_name,last_name,middle_name,email,password,'',phone_number,'','','','','','','users','active'))
                mysql.connection.commit()
                cursor.close()
                return redirect(url_for("login"))


        else :
            # return jsonify({"errors" : register_form.errors }), 400
            flash({"errors" : register_form.errors })
            return redirect(request.referrer)
    else :
        return render_template("auth/registartion.html")



In [16]:
if __name__== "__main__":
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [10/Jan/2025 15:47:05] "GET / HTTP/1.1" 302 -
127.0.0.1 - - [10/Jan/2025 15:47:05] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [10/Jan/2025 15:47:21] "POST /login HTTP/1.1" 200 -
